In [3]:
import nibabel as nib
import numpy as np

# Load the images
image1_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset501\\imagesTr\\ESTNEMC0013_1_frame0_198_0000.nii'
image2_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset905_SegmentOCT3d3\\imagesTr\\ESTNEMC0013_1_frame0_198_0000.nii.gz'
label1_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset501\\labelsTr\\ESTNEMC0013_1_frame0_198.nii'
label2_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset905_SegmentOCT3d3\\labelsTr\\ESTNEMC0013_1_frame0_198.nii.gz'

image1 = nib.load(image1_path)
image2 = nib.load(image2_path)

# Convert images to numpy arrays
image1_array = image1.get_fdata()
image2_array = image2.get_fdata()

label1 = nib.load(label1_path)
label2 = nib.load(label2_path)

# Convert images to numpy arrays
label1_array = label1.get_fdata()
label2_array = label2.get_fdata()

# Print the shapes of the images
print("Shape of image 1:", image1_array.shape)
print("Shape of image 2:", image2_array.shape)

print("Shape of image 1:", label1_array.shape)
print("Shape of image 2:", label2_array.shape)

Shape of image 1: (704, 704, 7)
Shape of image 2: (704, 704, 1)
Shape of image 1: (704, 704, 1)
Shape of image 2: (704, 704, 1)
